In [ ]:
# @title ## 6. Receptor File Parameter Analysis (Optional)
# @markdown ### Functionality Description
# @markdown This script is used to process protein receptor files (PDB format) with the following main functionalities:
# @markdown 1. Use PROPKA to predict the pKa values of amino acids.
# @markdown 2. Calculate the molecular weight and number of amino acids in the protein.
# @markdown 3. Generate statistical charts and data analysis.
# @markdown 4. Export the processing results and charts.
# @markdown
# @markdown ### Usage Instructions
# @markdown a. Please **click the run button on the left** and follow the prompts.

# @markdown b. **Upload one or more PDB files**, which will be processed automatically.

# @markdown c. After processing is complete, the processed files, charts, and data will be automatically packaged and a download link will be provided.

!/usr/bin/python3 -m pip install propka

try:
    from propka.run import single
    print("Successfully imported propka.")
except ImportError as e:
    print(f"Import failed: {e}")

from google.colab import files as colab_files
import sys
import subprocess
import os
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from tqdm import tqdm
import pandas as pd
import numpy as np
import traceback
import io
import zipfile
import re

try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
    print("Warning: This script is designed to run in the Google Colab environment. Some functions may not work properly.")

def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

required_packages = ['matplotlib', 'seaborn', 'tqdm', 'pandas', 'numpy', 'propka']

for package in required_packages:
    try:
        __import__(package)
    except ImportError:
        print(f"{package} is not installed, installing...")
        install(package)
        print(f"{package} installation complete.")

def calculate_molecular_properties(file_path):
    try:
        mol_weight = 0
        num_residues = 0
        with open(file_path, 'r') as f:
            for line in f:
                if line.startswith('ATOM'):
                    atom_mass = {
                        'C': 12.01, 'N': 14.01, 'O': 16.00, 'S': 32.07,
                        'H': 1.008, 'P': 30.97, 'F': 19.00, 'Cl': 35.45,
                        'Br': 79.90, 'I': 126.90
                    }
                    atom_type = line[76:78].strip()
                    mol_weight += atom_mass.get(atom_type, 0)
                    if line[17:20] != 'HOH':  # Skip water molecules
                        num_residues += 1
        num_residues = num_residues // 10  # Assume an average of 10 atoms per amino acid
        return mol_weight, num_residues
    except Exception as e:
        print(f"Error calculating molecular properties: {str(e)}")
        traceback.print_exc()
        return None, None

def predict_pka_states(file_path):
    try:
        molecule = single(file_path)

        if not hasattr(molecule, 'conformations') or len(molecule.conformations) == 0:
            return None

        first_conformation = list(molecule.conformations.values())[0]

        if not hasattr(first_conformation, 'groups'):
            return None

        groups = first_conformation.groups

        histidine_states = []
        all_residue_pkas = []

        for group in groups:
            if hasattr(group, 'type'):
                if group.type == 'HIS':
                    histidine_states.append(group.pka_value)
                if group.type in ['ASP', 'GLU', 'HIS', 'CYS', 'LYS', 'ARG', 'TYR']:
                    all_residue_pkas.append((group.type, group.pka_value))

        if not histidine_states and not all_residue_pkas:
            return None

        propka_output = io.StringIO()
        sys.stdout = propka_output
        molecule.write_pka()
        sys.stdout = sys.__stdout__
        propka_summary = propka_output.getvalue()

        return {
            'histidine': histidine_states,
            'all_residues': all_residue_pkas,
            'propka_summary': propka_summary
        }
    except Exception as e:
        print(f"Error predicting pKa states: {str(e)}")
        traceback.print_exc()
        return None

def visualize_results(results, output_dir):
    custom_colors = ['#a1dce9', '#d2eeef', '#96e7d3', '#dae4be', '#ecdae5',
                     '#938cbe', '#acb2da', '#8d9fc3', '#d2cad8', '#c7cbf6']

    marker_styles = ['o', 's', '^', 'D', 'X', 'P', '<', '>', 'H', 'v']

    num_files = len(results['histidine_states'])
    colors = custom_colors[:num_files] if num_files <= len(custom_colors) else sns.color_palette("husl", num_files)

    if results['success'] == 0:
        print("No files processed successfully, unable to generate visual results.")
        return

    file_names = [re.sub(r'\.pdb$', '', os.path.basename(f)) for f in results['file_names']]

    plt.figure(figsize=(10, 6))
    plt.pie([results['success'], results['failed']], labels=['Successful', 'Failed'], autopct='%1.1f%%', colors=custom_colors[:2])
    plt.title('Processing Result Statistics')
    plt.savefig(os.path.join(output_dir, 'processing_result_statistics.png'), dpi=300)
    plt.close()

    plt.figure(figsize=(10, 6))
    if results['histidine_states']:
        for i, histidine_pkas in enumerate(results['histidine_states']):
            label = file_names[i]
            sns.kdeplot(histidine_pkas, fill=True, label=label, color=colors[i])  # Continue using fill
        plt.title('Histidine pKa Distribution')
        plt.xlabel('pKa Value')
        plt.ylabel('Density')
        plt.legend()
    else:
        plt.text(0.5, 0.5, 'No Histidine pKa Data', ha='center', va='center')
    plt.savefig(os.path.join(output_dir, 'his_pka_distribution.png'), dpi=300)
    plt.close()

    plt.figure(figsize=(10, 6))
    all_histidine_pkas = [pka for sublist in results['histidine_states'] for pka in sublist]
    if all_histidine_pkas:
        sns.histplot(all_histidine_pkas, bins=20, kde=True, color=custom_colors[2])  # Use custom colors
        plt.title('Overall Histidine pKa Distribution')
        plt.xlabel('pKa Value')
        plt.ylabel('Frequency')
    else:
        plt.text(0.5, 0.5, 'No Histidine pKa Data', ha='center', va='center')
    plt.savefig(os.path.join(output_dir, 'overall_his_pka_distribution.png'), dpi=300)
    plt.close()

    plt.figure(figsize=(10, 6))
    if results['molecular_weights'] and results['num_residues']:
        for i, (mw, nr) in enumerate(zip(results['molecular_weights'], results['num_residues'])):
            label = file_names[i]
            plt.scatter(nr, mw, label=label, color=colors[i], marker=marker_styles[i % len(marker_styles)], edgecolor='k')  # Use different marker styles
        plt.title('Molecular Weight vs Number of Residues')
        plt.xlabel('Number of Residues')
        plt.ylabel('Molecular Weight (Da)')
        plt.legend()
    else:
        plt.text(0.5, 0.5, 'No Molecular Weight or Number of Residues Data', ha='center', va='center')
    plt.savefig(os.path.join(output_dir, 'molecular_weight_vs_amino_acid_number.png'), dpi=300)
    plt.close()

    plt.figure(figsize=(15, 8))
    if results['residue_pkas']:
        pka_df = pd.DataFrame(results['residue_pkas'], columns=['residue', 'pka', 'file'])
        pka_summary = pka_df.groupby('residue').agg(mean_pka=('pka', 'mean'), std_pka=('pka', 'std')).reset_index()
        pka_pivot = pka_summary.pivot(index='residue', columns='mean_pka', values='std_pka')
        sns.heatmap(pka_pivot, annot=True, fmt='.2f', cmap='YlGnBu')
        plt.title('Amino Acid pKa Distribution (Mean vs Std)')
        plt.xlabel('Mean pKa')
        plt.ylabel('Residue')
    else:
        plt.text(0.5, 0.5, 'No Amino Acid pKa Data', ha='center', va='center')
    plt.savefig(os.path.join(output_dir, 'amino_acid_pka_distribution.png'), dpi=300)
    plt.close()

    plt.figure(figsize=(15, 8))
    if results['residue_pkas']:
        pka_df = pd.DataFrame(results['residue_pkas'], columns=['residue', 'pka', 'file'])
        pka_matrix = pka_df.pivot_table(index='file', columns='residue', values='pka', aggfunc='mean')
        sns.heatmap(pka_matrix, annot=True, fmt='.2f', cmap='coolwarm', linewidths=.5)
        plt.title('Overall Amino Acid pKa Distribution')
        plt.xlabel('Residue')
        plt.ylabel('File')
    else:
        plt.text(0.5, 0.5, 'No Amino Acid pKa Data', ha='center', va='center')
    plt.savefig(os.path.join(output_dir, 'overall_amino_acid_pka_distribution.png'), dpi=300)
    plt.close()

    plt.figure(figsize=(10, 6))
    if results['residue_pkas']:
        pka_df['file'] = pka_df['file'].apply(lambda x: re.sub(r'\.pdb$', '', os.path.basename(x)))
        sns.boxplot(x='residue', y='pka', hue='file', data=pka_df, palette=colors)
        plt.title('Amino Acid pKa Box Plot')
        plt.xlabel('Amino Acid Type')
        plt.ylabel('pKa Value')
        plt.xticks(rotation=45)
        plt.legend()
    else:
        plt.text(0.5, 0.5, 'No Amino Acid pKa Data', ha='center', va='center')
    plt.savefig(os.path.join(output_dir, 'amino_acid_pka_box_plot.png'), dpi=300)
    plt.close()

    plt.figure(figsize=(10, 6))
    if results['molecular_weights'] and results['num_residues'] and results['histidine_states']:
        corr_data = pd.DataFrame({
            'Molecular Weight': results['molecular_weights'],
            'Number of Residues': results['num_residues'],
            'Average His pKa': [np.mean(his_pkas) if his_pkas else np.nan for his_pkas in results['histidine_states']]
        })
        sns.heatmap(corr_data.corr(), annot=True, cmap='RdYlBu')
        plt.title('Characteristic Correlation Heat Map')
    else:
        plt.text(0.5, 0.5, 'There is not enough data to generate a correlation heat map', ha='center', va='center')
    plt.savefig(os.path.join(output_dir, 'correlation_heatmap.png'), dpi=300)
    plt.close()

    if results['residue_pkas']:
        pka_df.to_csv(os.path.join(output_dir, 'residue_pkas.csv'), index=False)

    if results['molecular_weights'] and results['num_residues'] and results['histidine_states']:
        corr_data.to_csv(os.path.join(output_dir, 'correlation_data.csv'), index=False)

def process_receptor_files():
    if not IN_COLAB:
        print("This function is only available in the Google Colab environment.")
        return

    print("Please upload one or more receptor files (.pdb format)...")
    uploaded = colab_files.upload()

    if not uploaded:
        print("Error: No files uploaded. Please re-upload the receptor files.")
        return

    output_directory = 'receptor_analysis_results'
    os.makedirs(output_directory, exist_ok=True)

    results = {
        'success': 0,
        'failed': 0,
        'histidine_states': [],
        'residue_pkas': [],
        'molecular_weights': [],
        'num_residues': [],
        'file_names': [],  
        'file_results': {}  
    }

    log_file = io.StringIO()

    for filename, file_content in uploaded.items():
        print(f"\nProcessing file: {filename}", file=log_file)
        results['file_names'].append(filename)  

        try:
            temp_file_path = os.path.join(output_directory, filename)
            with open(temp_file_path, 'wb') as temp_file:
                temp_file.write(file_content)

            pka_results = predict_pka_states(temp_file_path)
            if pka_results is None:
                print(f"Warning: pKa prediction results for file {filename} are empty", file=log_file)
                results['failed'] += 1
                continue

            if pka_results['histidine'] or pka_results['all_residues']:
                results['histidine_states'].append(pka_results['histidine'])
                results['residue_pkas'].extend([(residue, pka, filename) for residue, pka in pka_results['all_residues']])
                print(f"Successfully predicted pKa states: {len(pka_results['histidine'])} histidines, {len(pka_results['all_residues'])} amino acids", file=log_file)

                with open(os.path.join(output_directory, f'{filename}_propka_summary.txt'), 'w') as f:
                    f.write(pka_results['propka_summary'])
            else:
                print(f"Warning: No valid amino acids found in file {filename}", file=log_file)
                results['failed'] += 1
                continue

            mol_weight, num_res = calculate_molecular_properties(temp_file_path)
            if mol_weight is not None and num_res is not None:
                results['molecular_weights'].append(mol_weight)
                results['num_residues'].append(num_res)
                print(f"Successfully calculated molecular properties: Molecular Weight = {mol_weight:.2f}, Number of Residues = {num_res}", file=log_file)
            else:
                print(f"Warning: Molecular property calculation for file {filename} failed", file=log_file)
                results['failed'] += 1
                continue

            results['success'] += 1
            print(f"File {filename} processed successfully", file=log_file)

            results['file_results'][filename] = {
                'histidine_states': pka_results['histidine'],
                'residue_pkas': pka_results['all_residues'],
                'molecular_weight': mol_weight,
                'num_residues': num_res
            }

        except Exception as e:
            results['failed'] += 1
            print(f"File {filename} processing failed: {str(e)}", file=log_file)
            traceback.print_exc(file=log_file)

        finally:
            if os.path.exists(temp_file_path):
                os.remove(temp_file_path)

    with open(os.path.join(output_directory, 'processing_log.txt'), 'w') as f:
        f.write(log_file.getvalue())

    visualize_results(results, output_directory)

    for filename, file_result in results['file_results'].items():
        with open(os.path.join(output_directory, f'{filename}_results.txt'), 'w') as f:
            f.write(f"File: {filename}\n")
            f.write(f"Molecular Weight: {file_result['molecular_weight']:.2f} Da\n")
            f.write(f"Number of Residues: {file_result['num_residues']}\n")
            f.write(f"Histidine pKa values: {', '.join(map(str, file_result['histidine_states']))}\n")
            f.write("Residue pKa values:\n")
            for residue, pka in file_result['residue_pkas']:
                f.write(f"  {residue}: {pka:.2f}\n")

    print("\nStatistics:")
    print(f"Total number of proteins processed: {results['success'] + results['failed']}")
    print(f"Success rate: {results['success'] / (results['success'] + results['failed']) * 100:.2f}%")

    if results['molecular_weights']:
        print(f"Average molecular weight: {np.mean(results['molecular_weights']):.2f} Da")
    else:
        print("No molecular weight data")

    if results['num_residues']:
        print(f"Average number of residues: {np.mean(results['num_residues']):.2f}")
    else:
        print("No number of residues data")

        if results['histidine_states']:
            print(f"Average histidine pKa: {np.mean([np.mean(pkas) for pkas in results['histidine_states'] if pkas]):.2f}")
        else:
            print("No histidine pKa data")

    with open(os.path.join(output_directory, 'summary_results.txt'), 'w') as f:
        f.write("Statistics:\n")
        f.write(f"Total number of proteins processed: {results['success'] + results['failed']}\n")
        f.write(f"Success rate: {results['success'] / (results['success'] + results['failed']) * 100:.2f}%\n")
        if results['molecular_weights']:
            f.write(f"Average molecular weight: {np.mean(results['molecular_weights']):.2f} Da\n")
        if results['num_residues']:
            f.write(f"Average number of residues: {np.mean(results['num_residues']):.2f}\n")
        if results['histidine_states']:
            f.write(f"Average histidine pKa: {np.mean([np.mean(pkas) for pkas in results['histidine_states'] if pkas]):.2f}\n")

    with zipfile.ZipFile('receptor_analysis_results.zip', 'w') as zipf:
        for root, dirs, files in os.walk(output_directory):
            for file in files:
                zipf.write(os.path.join(root, file),
                           os.path.relpath(os.path.join(root, file),
                                           os.path.join(output_directory, '..')))

    colab_files.download('receptor_analysis_results.zip')

process_receptor_files()